Импорт библиотек

In [1]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
import optuna
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

Загрузка данных

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
def objective(trial):
  sv_layers_cnt = trial.suggest_categorical('sv_layers_cnt', [1])
  sv_units = trial.suggest_categorical('sv_units', [32, 64, 128])
  layers_cnt = trial.suggest_categorical('layers_cnt', [1])
  units = trial.suggest_categorical('units', [32, 64, 128])
  reg_type = trial.suggest_categorical('reg_type', ['L1','L2'])
  reg_value = trial.suggest_float('reg_value', 0, 1, step=0.1)
  optimizer_name = trial.suggest_categorical('optimizer', ['adam','sgd'])
  learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
  patience = trial.suggest_categorical('patience', [10])

  input_layer = Input(shape=(32, 32, 3))

  for sv in range(sv_layers_cnt):
    x = Conv2D(sv_units, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)

  for l in range(layers_cnt):
    if reg_type == 'L1':
      x = Dense(units,
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l1(reg_value))(x)
    else:
      x = Dense(units,
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l2(reg_value))(x)

  output_layer = Dense(10, activation='softmax')(x)

  model = Model(inputs=input_layer, outputs=output_layer)

  if optimizer_name == 'adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizer_name =='sgd':
    optimizer = SGD(learning_rate=learning_rate)

  model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  early_stopping = EarlyStopping(monitor='val_loss',
                                 patience=patience,
                                 restore_best_weights=True)

  history = model.fit(x_train,
                      y_train,
                      epochs=20,
                      validation_data=(x_test, y_test),
                      verbose=0,
                      callbacks=[early_stopping])
  val_accuracy = history.history['val_accuracy'][-1]

  return val_accuracy

In [ ]:
study = optuna.create_study(direction='maximize')  # 'maximize' for accuracy, 'minimize' for loss
study.optimize(objective, n_trials=50)

# Print the best parameters and value
print("Best parameters found: ", study.best_params)
print("Best value found: {:.2f}%".format(study.best_value * 100))

[I 2023-09-06 14:27:09,374] A new study created in memory with name: no-name-68c0cba9-0cd4-4d7f-872b-af1b0626c4f0
[I 2023-09-06 14:37:49,175] Trial 0 finished with value: 0.30880001187324524 and parameters: {'sv_layers_cnt': 1, 'sv_units': 32, 'layers_cnt': 1, 'units': 128, 'reg_type': 'L2', 'reg_value': 0.6000000000000001, 'optimizer': 'adam', 'learning_rate': 0.006310231590824998, 'patience': 10}. Best is trial 0 with value: 0.30880001187324524.
[I 2023-09-06 14:46:39,389] Trial 1 finished with value: 0.10000000149011612 and parameters: {'sv_layers_cnt': 1, 'sv_units': 32, 'layers_cnt': 1, 'units': 32, 'reg_type': 'L1', 'reg_value': 0.30000000000000004, 'optimizer': 'sgd', 'learning_rate': 7.551691577574794e-05, 'patience': 10}. Best is trial 0 with value: 0.30880001187324524.
[I 2023-09-06 14:59:20,740] Trial 2 finished with value: 0.10000000149011612 and parameters: {'sv_layers_cnt': 1, 'sv_units': 32, 'layers_cnt': 1, 'units': 32, 'reg_type': 'L1', 'reg_value': 0.30000000000000004

In [4]:
def objective(trial):
  sv_layers_cnt = trial.suggest_categorical('sv_layers_cnt', [1])
  sv_units = trial.suggest_categorical('sv_units', [64])
  layers_cnt = trial.suggest_categorical('layers_cnt', [1])
  units = trial.suggest_categorical('units', [64])
  reg_type = trial.suggest_categorical('reg_type', ['L2'])
  reg_value = trial.suggest_float('reg_value', 0.8, 1, step=0.05)
  optimizer_name = trial.suggest_categorical('optimizer', ['adam'])
  learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
  patience = trial.suggest_categorical('patience', [10])

  input_layer = Input(shape=(32, 32, 3))

  for sv in range(sv_layers_cnt):
    x = Conv2D(sv_units, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)

  for l in range(layers_cnt):
    if reg_type == 'L1':
      x = Dense(units,
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l1(reg_value))(x)
    else:
      x = Dense(units,
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l2(reg_value))(x)

  output_layer = Dense(10, activation='softmax')(x)

  model = Model(inputs=input_layer, outputs=output_layer)

  if optimizer_name == 'adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizer_name =='sgd':
    optimizer = SGD(learning_rate=learning_rate)

  model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  early_stopping = EarlyStopping(monitor='val_loss',
                                 patience=patience,
                                 restore_best_weights=True)

  history = model.fit(x_train,
                      y_train,
                      epochs=20,
                      validation_data=(x_test, y_test),
                      verbose=0,
                      callbacks=[early_stopping])
  val_accuracy = history.history['val_accuracy'][-1]

  return val_accuracy

In [5]:
study = optuna.create_study(direction='maximize')  # 'maximize' for accuracy, 'minimize' for loss
study.optimize(objective, n_trials=10)

# Print the best parameters and value
print("Best parameters found: ", study.best_params)
print("Best value found: {:.2f}%".format(study.best_value * 100))

[I 2023-09-07 09:41:54,213] A new study created in memory with name: no-name-f0c46b6d-3258-433f-a332-4c7ed13422ea
[I 2023-09-07 10:07:18,479] Trial 0 finished with value: 0.37470000982284546 and parameters: {'sv_layers_cnt': 1, 'sv_units': 64, 'layers_cnt': 1, 'units': 64, 'reg_type': 'L2', 'reg_value': 1.0, 'optimizer': 'adam', 'learning_rate': 5.031485759645525e-05, 'patience': 10}. Best is trial 0 with value: 0.37470000982284546.
[I 2023-09-07 10:30:43,053] Trial 1 finished with value: 0.3790000081062317 and parameters: {'sv_layers_cnt': 1, 'sv_units': 64, 'layers_cnt': 1, 'units': 64, 'reg_type': 'L2', 'reg_value': 0.9, 'optimizer': 'adam', 'learning_rate': 0.002091336915322076, 'patience': 10}. Best is trial 1 with value: 0.3790000081062317.
[I 2023-09-07 10:55:43,724] Trial 2 finished with value: 0.32249999046325684 and parameters: {'sv_layers_cnt': 1, 'sv_units': 64, 'layers_cnt': 1, 'units': 64, 'reg_type': 'L2', 'reg_value': 1.0, 'optimizer': 'adam', 'learning_rate': 1.4004342

Best parameters found:  {'sv_layers_cnt': 1, 'sv_units': 64, 'layers_cnt': 1, 'units': 64, 'reg_type': 'L2', 'reg_value': 0.8, 'optimizer': 'adam', 'learning_rate': 0.0007868103936879903, 'patience': 10}
Best value found: 44.75%
